# Assignment Module 2: Product Classification

The goal of this assignment is to implement a neural network that classifies smartphone pictures of products found in grocery stores. The assignment will be divided into two parts: first, you will be asked to implement from scratch your own neural network for image classification; then, you will fine-tune a pretrained network provided by PyTorch.


## Preliminaries: the dataset

The dataset you will be using contains natural images of products taken with a smartphone camera in different grocery stores:

<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Granny-Smith.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Pink-Lady.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Lemon.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Banana.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Vine-Tomato.jpg" width="150">
</p>
<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Yellow-Onion.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Green-Bell-Pepper.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Arla-Standard-Milk.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Oatly-Natural-Oatghurt.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Alpro-Fresh-Soy-Milk.jpg" width="150">
</p>

The products belong to the following 43 classes:
```
0.  Apple
1.  Avocado
2.  Banana
3.  Kiwi
4.  Lemon
5.  Lime
6.  Mango
7.  Melon
8.  Nectarine
9.  Orange
10. Papaya
11. Passion-Fruit
12. Peach
13. Pear
14. Pineapple
15. Plum
16. Pomegranate
17. Red-Grapefruit
18. Satsumas
19. Juice
20. Milk
21. Oatghurt
22. Oat-Milk
23. Sour-Cream
24. Sour-Milk
25. Soyghurt
26. Soy-Milk
27. Yoghurt
28. Asparagus
29. Aubergine
30. Cabbage
31. Carrots
32. Cucumber
33. Garlic
34. Ginger
35. Leek
36. Mushroom
37. Onion
38. Pepper
39. Potato
40. Red-Beet
41. Tomato
42. Zucchini
```

The dataset is split into training (`train`), validation (`val`), and test (`test`) set.

The following code cells download the dataset and define a `torch.utils.data.Dataset` class to access it. This `Dataset` class will be the starting point of your assignment: use it in your own code and build everything else around it.

In [12]:
#!git clone https://github.com/marcusklasson/GroceryStoreDataset.git

In [29]:
from pathlib import Path
from PIL import Image
from typing import List, Tuple

import torch
from torch import Tensor,device
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam, SGD, AdamW
import torch.nn.functional as F
import torch.nn as nn
import torchvision

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np

import random
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")

Device: cuda


In [14]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(45)

In [15]:
class GroceryStoreDataset(Dataset):

    def __init__(self, split: str, transform=None) -> None:
        super().__init__()

        self.root = Path("GroceryStoreDataset/dataset")
        self.split = split
        self.paths, self.labels = self.read_file()

        self.transform = transform

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:
        img = Image.open(self.root / self.paths[idx])
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

    def read_file(self) -> Tuple[List[str], List[int]]:
        paths = []
        labels = []

        with open(self.root / f"{self.split}.txt") as f:
            for line in f:
                # path, fine-grained class, coarse-grained class
                path, _, label = line.replace("\n", "").split(", ")
                paths.append(path), labels.append(int(label))

        return paths, labels

    def get_num_classes(self) -> int:
        return max(self.labels) + 1

In [16]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
train_dataset = GroceryStoreDataset(split='train',transform=transform)
val_dataset = GroceryStoreDataset(split='val',transform=transform)
test_dataset = GroceryStoreDataset(split='test',transform=transform)
train_dataset.__len__(),val_dataset.__len__(),test_dataset.__len__()

(2640, 296, 2485)

In [17]:
def check_image_shapes(dataset):
    for i in range(dataset.__len__()):
        img, label = dataset[i]
        w, h = 0,0
        #print(f"Image {i} shape: {img.shape} (width, height)")
        if img.shape[1] > w:
          w = img.shape[1]
        if img.shape[2] > h:
          h = img.shape[2]
    return w,h
# Check shapes of the first few images
w_t,h_t = check_image_shapes(train_dataset)
w_t,h_t
#w_v,h_v = check_image_shapes(val_dataset)
#w_T,h_T = check_image_shapes(test_dataset)

(464, 348)

In [52]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
image_height, image_width, image_channels = 348,348,3
# Define the transform to resize images
resize_transform = transforms.Compose([
    transforms.Resize((image_height, image_width)),
    transforms.ToTensor()
])

# Load the training dataset with the resizing transform
train_dataset = GroceryStoreDataset(split='train',transform=resize_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

# Initialize variables to store the mean and std
mean = torch.zeros(3)
std = torch.zeros(3)
nb_samples = 0

# Compute mean and standard deviation
for images, _ in train_loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print(f'Mean: {mean}')
print(f'Std: {std}')


Mean: tensor([0.5306, 0.3964, 0.2564])
Std: tensor([0.2329, 0.2094, 0.1785])


## Part 1: design your own network

Your goal is to implement a convolutional neural network for image classification and train it on `GroceryStoreDataset`. You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split of **around 60%**. You are free to achieve that however you want, except for a few rules you must follow:

- You **cannot** simply instantiate an off-the-self PyTorch network. Instead, you must construct your network as a composition of existing PyTorch layers. In more concrete terms, you can use e.g. `torch.nn.Linear`, but you **cannot** use e.g. `torchvision.models.alexnet`.

- Justify every *design choice* you make. Design choices include network architecture, training hyperparameters, and, possibly, dataset preprocessing steps. You can either (i) start from the simplest convolutional network you can think of and add complexity one step at a time, while showing how each step gets you closer to the target ~60%, or (ii) start from a model that is already able to achieve the desired accuracy and show how, by removing some of its components, its performance drops (i.e. an *ablation study*). You can *show* your results/improvements however you want: training plots, console-printed values or tables, or whatever else your heart desires: the clearer, the better.

Don't be too concerned with your network performance: the ~60% is just to give you an idea of when to stop. Keep in mind that a thoroughly justified model with lower accuracy will be rewarded **more** points than a poorly experimentally validated model with higher accuracy.

In [32]:
def ncorrect(scores, y):
    y_hat = torch.argmax(scores, -1)
    return (y_hat == y).sum()

def accuracy(scores, y):
    correct = ncorrect(scores, y)
    return correct.true_divide(y.shape[0])

def train_loop(model, train_dl, epochs, opt, val_dl=None, verbose=False, label_smoothing=0):
    best_val_acc = 0
    best_params = []
    best_epoch = -1
    found = False
    for e in tqdm(range(epochs)):
        model.train()
        # Train
        train_loss = 0
        train_samples = 0
        train_acc = 0
        for train_data in train_dl:
            imgs = train_data[0].to(device)
            labels = train_data[1].to(device)
            scores = model(imgs)
            loss = F.cross_entropy(scores, labels,label_smoothing=label_smoothing)
            train_loss += loss.item() * imgs.shape[0]
            train_samples += imgs.shape[0]
            train_acc += ncorrect(scores, labels).item()

            opt.zero_grad()  # clear
            loss.backward()  # fill
            opt.step()       # use

        train_acc /= train_samples
        train_loss /= train_samples

        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_samples = 0
            val_acc = 0
            if val_dl is not None:
                for val_data in val_dl:
                    imgs = val_data[0].to(device)
                    labels = val_data[1].to(device)
                    val_scores = model(imgs)
                    val_loss += F.cross_entropy(val_scores, labels).item() * imgs.shape[0]
                    val_samples += imgs.shape[0]
                    val_acc += ncorrect(val_scores, labels).item()
                val_acc /= val_samples
                val_loss /= val_samples

            if val_dl is None or val_acc > best_val_acc:
                best_val_acc = val_acc if val_dl is not None else 0
                best_params = model.state_dict()
                torch.save(best_params, "best_model.pth")
                best_epoch = e

        if verbose: #and e % 5 == 0:
            print(f"Epoch {e}: train loss {train_loss:.3f} - train acc {train_acc:.3f}" + ("" if val_dl is None else f" - valid loss {val_loss:.3f} - valid acc {val_acc:.3f}"))
            if val_acc >= 0.60:
                best_params = model.state_dict()
                torch.save(best_params, "A2.pth")
                best_epoch = e
                break

    if verbose and val_dl is not None:
        print(f"Best epoch {best_epoch}, best acc {best_val_acc}")

    return (best_val_acc, best_params, best_epoch)

# CNN

In [53]:
image_height, image_width, image_channels = 348,348,3 #224, 224, 3
num_classes = train_dataset.get_num_classes()

transform_2 = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(size=(image_height, image_width),antialias=True),
    #torchvision.transforms.Resize((image_height, image_width)),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std)
    #torchvision.transforms.Normalize((0.5, 0.4, 0.25), (0.23, 0.2, 0.17))
])

val_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((image_height, image_width)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std)
    #torchvision.transforms.transforms.Normalize((0.5, 0.4, 0.25), (0.23, 0.2, 0.17))
])
train_dataset = GroceryStoreDataset(split='train',transform=transform_2)
val_dataset = GroceryStoreDataset(split='val',transform=val_transforms)
#test_dataset = GroceryStoreDataset(split='test',transform=transform_2)

batch = 8
train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=False)

In [56]:
class ExtendedCNN(nn.Module):
    def __init__(self,channels=16, num_classes=num_classes,p_dropout=0):
        super(ExtendedCNN, self).__init__()
        # Define convolutional layers
        self.conv1 = nn.Conv2d(in_channels=image_channels, out_channels=channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=channels, out_channels=channels*2, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=channels*2, out_channels=channels*4, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=channels*4, out_channels=channels*8, kernel_size=3, stride=1, padding=1)

        # Calculate the size of the feature map after the convolutional layers
        def conv2d_size_out(size, kernel_size=3, stride=1, padding=1):
            return (size - kernel_size + 2 * padding) // stride + 1
        def conv2d_compute_size(conv_number,kernels,max_poolings):  #max pooling ex. [1,0,1] if 1 them there is a max_pool
          image_w = image_width
          for i in range(conv_number):
            image_w = conv2d_size_out(image_w, kernel_size=kernels[i])
            if max_poolings[i] == 1:
              image_w = image_w//2
          return image_w

        #convh = conv2d_size_out(conv2d_size_out(image_height, kernel_size=5, padding=1) // 2, kernel_size=3, padding=1) // 2
        image_w = conv2d_compute_size(4,[3,3,3,3],[1,1,1,1])

        linear_input_size = image_w * image_w * channels*8  # Adjusted based on the output channels of conv2

        # Define fully connected layers
        self.fc1 = nn.Linear(linear_input_size, 128)
        #self.drop1 = nn.Dropout(p_dropout)
        self.fc2 = nn.Linear(128, 64)
        #self.drop2 = nn.Dropout(p_dropout)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        # Apply convolutional layers with ReLU activation and max pooling
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        # Flatten the tensor
        x = x.view(x.size(0), -1)

        # Apply fully connected layers
        x = F.relu(self.fc1(x))
        #x = self.drop1(x)
        x = F.relu(self.fc2(x))
        #x = self.drop2(x)
        x = self.fc3(x)

        return x


In [57]:
new_model = ExtendedCNN(
    num_classes = num_classes,
)
new_model.to(device)

#Training with: weight_decay=0.0, label_smoothing=0.5, lr=0.001, channels=16, batch_size=8
    
#optimizer = torch.optim.AdamW(new_model.parameters(), lr=0.001,weight_decay=0.01)
optimizer = torch.optim.Adam(new_model.parameters(), lr=0.001)

epochs = 80
label_smoothing = 0.1
best_val_acc, best_params, best_epoch = train_loop(
    new_model,
    train_loader,
    epochs,
    optimizer,
    val_loader,
    verbose=True,
    label_smoothing = label_smoothing,
)

  0%|          | 0/80 [00:00<?, ?it/s]

Epoch 0: train loss 3.243 - train acc 0.162 - valid loss 3.005 - valid acc 0.186
Epoch 1: train loss 2.826 - train acc 0.243 - valid loss 2.623 - valid acc 0.277
Epoch 2: train loss 2.589 - train acc 0.311 - valid loss 2.867 - valid acc 0.209
Epoch 3: train loss 2.363 - train acc 0.385 - valid loss 2.228 - valid acc 0.294
Epoch 4: train loss 2.179 - train acc 0.447 - valid loss 2.199 - valid acc 0.304
Epoch 5: train loss 2.089 - train acc 0.482 - valid loss 1.878 - valid acc 0.453
Epoch 6: train loss 1.953 - train acc 0.520 - valid loss 2.026 - valid acc 0.389
Epoch 7: train loss 1.878 - train acc 0.556 - valid loss 2.066 - valid acc 0.382
Epoch 8: train loss 1.832 - train acc 0.569 - valid loss 1.845 - valid acc 0.432
Epoch 9: train loss 1.753 - train acc 0.599 - valid loss 1.900 - valid acc 0.429
Epoch 10: train loss 1.714 - train acc 0.616 - valid loss 1.720 - valid acc 0.439
Epoch 11: train loss 1.665 - train acc 0.639 - valid loss 2.097 - valid acc 0.446
Epoch 12: train loss 1.628

# Gridsearch

In [ ]:
weight_decays = [0.0,0.01,0.001]
label_smoothings = [0.0, 0.05, 0.1, 0.15, 0.2]
lrs = [0.001]#, 0.0001]
channels = [16]
batches = [8]
#p_dropout = [0.1,0.3,0.5]

train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=False)

epochs = 80
test_iteration = 1
total_test = len(weight_decays)*len(label_smoothings)*len(lrs)*len(channels)*len(batches)#*len(p_dropout)
#Training with: weight_decay=0.0, label_smoothing=0.5, lr=0.001, channels=16, batch_size=8

for weight_decay in weight_decays:
    for label_smoothing in label_smoothings:
        for lr in lrs:
            for channel in channels:
                for batch in batches:
                    model = ExtendedCNN(channels=channel, num_classes=num_classes,p_dropout=0)
                    model.to(device)

                    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

                    print(f"Training with: weight_decay={weight_decay}, label_smoothing={label_smoothing}, lr={lr}, channels={channel}, batch_size={batch},test={test_iteration}/{total_test}")
                    test_iteration+=1
                    best_val_acc, best_params, best_epoch, found = train_loop(
                        model,
                        train_loader,
                        epochs,
                        optimizer,
                        val_loader,
                        verbose=True,
                        label_smoothing = label_smoothing,
                    )

Training with: weight_decay=0.0, label_smoothing=0.0, lr=0.001, channels=16, batch_size=8,test=1/15


  0%|          | 0/80 [00:00<?, ?it/s]

Epoch 0: train loss 3.075 - train acc 0.169 - valid loss 3.038 - valid acc 0.226
Epoch 1: train loss 2.540 - train acc 0.265 - valid loss 2.808 - valid acc 0.233
Epoch 2: train loss 2.232 - train acc 0.309 - valid loss 2.427 - valid acc 0.307
Epoch 3: train loss 2.011 - train acc 0.379 - valid loss 2.641 - valid acc 0.304
Epoch 4: train loss 1.806 - train acc 0.431 - valid loss 2.472 - valid acc 0.291
Epoch 5: train loss 1.666 - train acc 0.482 - valid loss 2.491 - valid acc 0.331
Epoch 6: train loss 1.485 - train acc 0.521 - valid loss 1.926 - valid acc 0.426
Epoch 7: train loss 1.409 - train acc 0.548 - valid loss 2.087 - valid acc 0.409
Epoch 8: train loss 1.315 - train acc 0.575 - valid loss 2.044 - valid acc 0.402
Epoch 9: train loss 1.234 - train acc 0.600 - valid loss 2.141 - valid acc 0.412
Epoch 10: train loss 1.166 - train acc 0.627 - valid loss 2.490 - valid acc 0.382
Epoch 11: train loss 1.075 - train acc 0.644 - valid loss 2.291 - valid acc 0.395
Epoch 12: train loss 1.022

  0%|          | 0/80 [00:00<?, ?it/s]

Epoch 0: train loss 3.067 - train acc 0.192 - valid loss 2.784 - valid acc 0.226
Epoch 1: train loss 2.547 - train acc 0.300 - valid loss 2.722 - valid acc 0.291
Epoch 2: train loss 2.211 - train acc 0.386 - valid loss 2.564 - valid acc 0.277
Epoch 3: train loss 1.971 - train acc 0.459 - valid loss 2.137 - valid acc 0.341
Epoch 4: train loss 1.800 - train acc 0.509 - valid loss 1.999 - valid acc 0.395
Epoch 5: train loss 1.682 - train acc 0.550 - valid loss 2.110 - valid acc 0.382
Epoch 6: train loss 1.620 - train acc 0.564 - valid loss 2.032 - valid acc 0.429
Epoch 7: train loss 1.488 - train acc 0.618 - valid loss 1.979 - valid acc 0.382
Epoch 8: train loss 1.455 - train acc 0.631 - valid loss 2.042 - valid acc 0.426
Epoch 9: train loss 1.375 - train acc 0.660 - valid loss 1.821 - valid acc 0.432
Epoch 10: train loss 1.356 - train acc 0.659 - valid loss 1.926 - valid acc 0.412
Epoch 11: train loss 1.304 - train acc 0.698 - valid loss 1.837 - valid acc 0.483
Epoch 12: train loss 1.280

...

  0%|          | 0/80 [00:00<?, ?it/s]

Epoch 0: train loss 3.307 - train acc 0.156 - valid loss 3.168 - valid acc 0.139
Epoch 1: train loss 2.998 - train acc 0.231 - valid loss 2.830 - valid acc 0.267
Epoch 2: train loss 2.832 - train acc 0.291 - valid loss 2.681 - valid acc 0.236
Epoch 3: train loss 2.691 - train acc 0.330 - valid loss 2.573 - valid acc 0.301
Epoch 4: train loss 2.551 - train acc 0.378 - valid loss 2.537 - valid acc 0.284
Epoch 5: train loss 2.458 - train acc 0.401 - valid loss 2.333 - valid acc 0.314
Epoch 6: train loss 2.363 - train acc 0.450 - valid loss 2.563 - valid acc 0.247
Epoch 7: train loss 2.299 - train acc 0.461 - valid loss 2.206 - valid acc 0.345
Epoch 8: train loss 2.219 - train acc 0.488 - valid loss 2.495 - valid acc 0.297
Epoch 9: train loss 2.142 - train acc 0.514 - valid loss 2.166 - valid acc 0.361
Epoch 10: train loss 2.124 - train acc 0.531 - valid loss 2.111 - valid acc 0.372
Epoch 11: train loss 2.064 - train acc 0.555 - valid loss 2.075 - valid acc 0.382
Epoch 12: train loss 2.044

  0%|          | 0/80 [00:00<?, ?it/s]

Epoch 0: train loss 3.322 - train acc 0.178 - valid loss 3.073 - valid acc 0.179
Epoch 1: train loss 3.055 - train acc 0.237 - valid loss 2.793 - valid acc 0.236
Epoch 2: train loss 2.855 - train acc 0.312 - valid loss 2.577 - valid acc 0.311
Epoch 3: train loss 2.713 - train acc 0.381 - valid loss 2.453 - valid acc 0.324
Epoch 4: train loss 2.619 - train acc 0.403 - valid loss 2.315 - valid acc 0.345
Epoch 5: train loss 2.509 - train acc 0.438 - valid loss 2.332 - valid acc 0.331
Epoch 6: train loss 2.447 - train acc 0.466 - valid loss 2.345 - valid acc 0.338
Epoch 7: train loss 2.389 - train acc 0.494 - valid loss 2.191 - valid acc 0.429
Epoch 8: train loss 2.328 - train acc 0.516 - valid loss 2.305 - valid acc 0.358
Epoch 9: train loss 2.292 - train acc 0.540 - valid loss 2.263 - valid acc 0.321
Epoch 10: train loss 2.284 - train acc 0.549 - valid loss 2.143 - valid acc 0.351
Epoch 11: train loss 2.212 - train acc 0.569 - valid loss 2.141 - valid acc 0.361
Epoch 12: train loss 2.187

KeyboardInterrupt: 